In [1]:
import pandas as pd
import numpy as np

import os, shutil, argparse

In [2]:
df = pd.read_csv("pitch_project_data.csv")
df.head()

,bcubeID,year,teamName,league,level,levelID,orgName,firstName,lastName,fullName,...,mlb_binary,milb_exp,ba100_post,org_top10_post,sign_bonus_mill,cpi_factor,sign_bonus_mill_cpi_adj,college,bloodlines,draft_round
0,25,2000,Queens,NYPL,A-,2,TOR,Brandon,Lyon,Brandon Lyon,...,1,1,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14
1,25,2001,Tennessee,SOU,AA,5,TOR,Brandon,Lyon,Brandon Lyon,...,1,2,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14
2,25,2001,Syracuse,IL,AAA,6,TOR,Brandon,Lyon,Brandon Lyon,...,1,2,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14
3,25,2001,Blue Jays,MLB,MLB,7,Blue Jays,Brandon,Lyon,Brandon Lyon,...,1,2,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14
4,25,2002,Syracuse,IL,AAA,6,TOR,Brandon,Lyon,Brandon Lyon,...,1,3,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14


In [3]:
df.columns

Index(['bcubeID', 'year', 'teamName', 'league', 'level', 'levelID', 'orgName',
       'firstName', 'lastName', 'fullName', 'wins', 'loss', 'gm', 'gs', 'cg',
       'sho', 'gf', 'sv', 'ip', 'hits', 'hr', 'runs', 'er', 'bb', 'ibb', 'so',
       'wp', 'bk', 'era', 'hits9', 'hr9', 'bb9', 'so9', 'so_bb', 'whip',
       'gs_pct', 'ip_gm', 'rel_app', 'hgt_inches', 'wgt', 'bats', 'throws',
       'dob', 'milb_debut', 'mlb_binary', 'milb_exp', 'ba100_post',
       'org_top10_post', 'sign_bonus_mill', 'cpi_factor',
       'sign_bonus_mill_cpi_adj', 'college', 'bloodlines', 'draft_round'],
      dtype='object')

In [4]:
expl = pd.read_csv("project_features.csv", header=None).transpose()
expl.columns = expl.iloc[0]
expl = expl[1:]
expl.head()

,bcubeID,year,teamName,league,level,levelID,orgName,firstName,lastName,fullName,...,milb_debut,mlb_binary,milb_exp,ba100_post,org_top10_post,sign_bonus_mill,sign_bonus_mill_cpi_adj,college,bloodlines,draft_round
1,unique identifier,season,team name,league stats are generated in,level of competition,numeric identifier for level,"MLB organization ""teamName"" is affliated with",player's first name,player's last name,player's full name,...,year of professional baseball debut (MILB / MLB),1 if appeared in MLB / 0 otherwise,year of professional experience --> (year - mi...,1 if Baseball America top 100 prospect / 0 oth...,1 if top 10 prosepct in respective organizatio...,signing bonus in millions of dollars,signing bonus in millions of dollars adjusted ...,college played at,relatives of player that played professional b...,draft round --> set to 9999 if not drafted or ...


In [5]:
sum(df[df['year']==2018]['org_top10_post']==1) # verifies that rankings in 2018 are all 0

0

In [6]:
df2017 = df[df['year']==2017]
print("2017 Top 10 Prospect List:", sum(df[df['year']==2017]['org_top10_post']==1), "players")
print("Unique Players in 2017 Top 10 List:", len(set(list(df2017[df2017['org_top10_post']==1].bcubeID))), "players")
df2017 = df2017[df2017['org_top10_post']==1];
print("Levels players in top10 played in 2017", len(set(list(df2017.level))))
print()
print("Number of players in each league")
print(df2017['level'].value_counts())
print(set(list(df2017.level)))
print('No need to distinguish between levels')
print(df2017['orgName'].value_counts())
pl = list(df2017.bcubeID); org = set(list(df2017.orgName))
for o in sorted(org):
    print(o, len(set(df2017[df2017.orgName == o].bcubeID)))

2017 Top 10 Prospect List: 237 players
Unique Players in 2017 Top 10 List: 125 players
Levels players in top10 played in 2017 7

Number of players in each league
A+     54
AA     50
Rk     34
A      34
AAA    28
A-     20
MLB    17
Name: level, dtype: int64
{'AAA', 'A-', 'MLB', 'A+', 'A', 'AA', 'Rk'}
No need to distinguish between levels
HOU             15
NYA             13
CHN             12
SDN             11
ATL             10
SLN             10
PIT              9
MIN              9
NYN              9
LAN              8
CHA              8
BOS              7
PHI              7
BAL              7
KCA              7
WAS              6
ARI              6
OAK              6
MIA              6
DET              6
SFN              5
CIN              5
TOR              5
SEA              5
ANA              5
COL              5
MIL              5
TEX              5
CLE              4
TBA              4
Cardinals        2
Braves           2
Astros           1
Royals           1
Reds          

## Problem 1: Integrate Non-unique players under one metric

In [7]:
pl = list(df.bcubeID); # Player IDs
p = pl[0]
mod = df[df.bcubeID==p]
mod = mod[df.year==2005]
mod.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,bcubeID,year,teamName,league,level,levelID,orgName,firstName,lastName,fullName,...,mlb_binary,milb_exp,ba100_post,org_top10_post,sign_bonus_mill,cpi_factor,sign_bonus_mill_cpi_adj,college,bloodlines,draft_round
9,25,2005,Tucson,PCL,AAA,6,ARI,Brandon,Lyon,Brandon Lyon,...,1,6,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14
10,25,2005,Diamondbacks,MLB,MLB,7,Diamondbacks,Brandon,Lyon,Brandon Lyon,...,1,6,0,0,NaN,1.46,NaN,"Dixie State College of Utah (St. George,UT)",NaN,14


In [8]:
df.columns

Index(['bcubeID', 'year', 'teamName', 'league', 'level', 'levelID', 'orgName',
       'firstName', 'lastName', 'fullName', 'wins', 'loss', 'gm', 'gs', 'cg',
       'sho', 'gf', 'sv', 'ip', 'hits', 'hr', 'runs', 'er', 'bb', 'ibb', 'so',
       'wp', 'bk', 'era', 'hits9', 'hr9', 'bb9', 'so9', 'so_bb', 'whip',
       'gs_pct', 'ip_gm', 'rel_app', 'hgt_inches', 'wgt', 'bats', 'throws',
       'dob', 'milb_debut', 'mlb_binary', 'milb_exp', 'ba100_post',
       'org_top10_post', 'sign_bonus_mill', 'cpi_factor',
       'sign_bonus_mill_cpi_adj', 'college', 'bloodlines', 'draft_round'],
      dtype='object')

In [10]:
expl['win']

KeyError: 'win'